In [1]:
from main  import *
text = 'C:/Users/ASUS/AppData/Local/Temp/tmpa70zfck3.py'

vlayer = QgsVectorLayer(wards_shapefile, "airports", "ogr")
vlayer2 = QgsVectorLayer(lga_shapefile, "SHEET", "ogr")
vlayer3 = QgsVectorLayer(set_extent_shapefile, "SHEET", "ogr")

# for field in vlayer.fields():
#     print(field.name(), field.typeName())
# vlayer.displayField()


def geo_location(wards_layer,extent, state,lga,ward):
    query = f'"statename"  =  \'{state}\' AND  "lganame"  =  \'{kwara_lga_map[lga]}\'AND "wardname" = \'{kwara_wards_map[ward]}\''
    ward_layer= crt_subset_lyr(wards_layer, query)
    ward_extent  = clip_set_ext_layer(extent,ward_layer)
    return create_random_point(ward_extent)



1
[<qgis._core.QgsFeature object at 0x000001FA6C875090>, <qgis._core.QgsFeature object at 0x000001FA6C874F70>, <qgis._core.QgsFeature object at 0x000001FA6C874E50>, <qgis._core.QgsFeature object at 0x000001FA6C875120>, <qgis._core.QgsFeature object at 0x000001FA6C874D30>, <qgis._core.QgsFeature object at 0x000001FA6C8751B0>, <qgis._core.QgsFeature object at 0x000001FA6C875240>, <qgis._core.QgsFeature object at 0x000001FA6C8752D0>, <qgis._core.QgsFeature object at 0x000001FA6C875360>, <qgis._core.QgsFeature object at 0x000001FA6C8753F0>, <qgis._core.QgsFeature object at 0x000001FA6C875480>, <qgis._core.QgsFeature object at 0x000001FA6C875510>, <qgis._core.QgsFeature object at 0x000001FA6C8755A0>, <qgis._core.QgsFeature object at 0x000001FA6C875630>, <qgis._core.QgsFeature object at 0x000001FA6C8756C0>, <qgis._core.QgsFeature object at 0x000001FA6C875750>, <qgis._core.QgsFeature object at 0x000001FA6C874CA0>, <qgis._core.QgsFeature object at 0x000001FA6C874DC0>, <qgis._core.QgsFeature ob

In [ ]:
# processing.algorithmHelp("native:selectbylocation")
# parameters = { 'INPUT' : state_extent, 
#                'VALUE' : 1, 
#                 }
# point =processing.run('native:selectbylocation', parameters)
# /algs = dict()
# for alg in QgsApplication.processingRegistry().algorithms():
#     algs[alg.displayName()] = alg.id()
#     print(alg.displayName(), alg.id() )